In [1]:
from EduNLP.Pretrain import TokenizerForHuggingface

/data/qlh/anaconda3/envs/py36/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# EduNLP 通用化接口测试

In [2]:
tokenizer = TokenizerForHuggingface("bert-base-chinese", add_specials=True, tokenize_method="ast_formula")

## 基础用法

In [3]:
text = '有公式$\\FormFigureID{wrong1?}$和公式$\\FormFigureBase64{wrong2?}$，\
                    如图$\\FigureID{088f15ea-8b7c-11eb-897e-b46bfc50aa29}$,\
                    若$x,y$满足约束条件$\\SIFSep$，则$z=x+7 y$的最大值为$\\SIFBlank$'

tokens = tokenizer.tokenize(text)
print(tokens)


encodes = tokenizer.bert_tokenizer(text)
print(encodes)

['公', '式', '[FORMULA]', '公', '式', '[FORMULA]', '如', '图', '[FIGURE]', 'ma', '##th', '##or', '##d', '_', '0', ',', 'ma', '##th', '##or', '##d', '_', '1', '约', '束', '条', '件', '[SEP]', 'ma', '##th', '##or', '##d', '_', '2', '=', 'ma', '##th', '##or', '##d', '_', '0', '+', 'text', '##or', '##d', 'ma', '##th', '##or', '##d', '_', '1', '最', '大', '值', '[MARK]']
{'input_ids': [101, 3300, 1062, 2466, 109, 139, 100, 169, 165, 8991, 8181, 8148, 136, 171, 109, 1469, 1062, 2466, 109, 139, 100, 169, 165, 8991, 8181, 8144, 136, 171, 109, 8024, 1963, 1745, 109, 139, 100, 169, 8142, 8156, 8189, 8493, 10073, 118, 129, 8204, 8161, 8177, 118, 8111, 8154, 8204, 118, 8747, 8154, 118, 144, 9340, 8204, 9717, 8474, 10226, 8887, 171, 109, 117, 5735, 109, 166, 117, 167, 109, 4007, 6639, 5276, 3338, 3340, 816, 109, 139, 100, 109, 8024, 1156, 109, 168, 134, 166, 116, 128, 167, 109, 4638, 3297, 1920, 966, 711, 109, 139, 100, 109, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 实现步骤

In [4]:
pre_tokens = tokenizer._pre_tokenize(text)
print(pre_tokens)

print(tokenizer.bert_tokenizer.tokenize(pre_tokens))

公式 [FORMULA] 公式 [FORMULA] 如图 [FIGURE] mathord_0 , mathord_1 约束条件 [SEP] mathord_2 = mathord_0 + textord mathord_1 最大值 [MARK]
['公', '式', '[FORMULA]', '公', '式', '[FORMULA]', '如', '图', '[FIGURE]', 'ma', '##th', '##or', '##d', '_', '0', ',', 'ma', '##th', '##or', '##d', '_', '1', '约', '束', '条', '件', '[SEP]', 'ma', '##th', '##or', '##d', '_', '2', '=', 'ma', '##th', '##or', '##d', '_', '0', '+', 'text', '##or', '##d', 'ma', '##th', '##or', '##d', '_', '1', '最', '大', '值', '[MARK]']


## 扩充词表

In [5]:
tokenizer.bert_tokenizer.encode("[FIGURE]")

[101, 21130, 102]

In [6]:
print(tokenizer._special_tokens)

{'[TAG]', '[FORMULA_END]', '[TEXT]', '[FORMULA_BEGIN]', '[FORMULA]', '[TEXT_BEGIN]', '[TEXT_END]', '[FIGURE]', '[MARK]', '[SEP]'}


### 直接新增单词

In [7]:
tokenizer.add_tokens(["[python]"])
tokenizer.bert_tokenizer.tokenize("[python] is a coding language")

['[python]', 'is', 'a', 'co', '##ding', 'language']

### 批量设置语料库词表

In [8]:
vocab_sentences = [
   '有公式$\\FormFigureID{wrong1?}$和公式$\\FormFigureBase64{wrong2?}$，如图$\\FigureID{088f15ea-8b7c-11eb-897e-b46bfc50aa29}$, 若$x,y$满足约束条件$\\SIFSep$，则$z=x+7 y$的最大值为$\\SIFBlank$'
]

added_words, added_num = tokenizer.set_vocab(vocab_sentences, lower=False, trim_min_count=1, do_tokenize=True)

print(added_words)

['公式', '[FORMULA]', '如图', '[FIGURE]', 'mathord_0', ',', 'mathord_1', '约束条件', '[SEP]', 'mathord_2', '=', '+', 'textord', '最大值', '[MARK]']


In [9]:
vocab_tokens = [
   ['公式', '[FORMULA]', '公式', '[FORMULA]', '如图', '[FIGURE]', 'mathord_0', ',', 'mathord_1', '约束条件', '[SEP]', 'mathord_2', '=', 'mathord_0', '+', 'textord', 'mathord_1', '最大值', '[MARK]']
]

added_words, added_num = tokenizer.set_vocab(vocab_tokens, lower=False, trim_min_count=1, do_tokenize=False)

print(added_words)

['公式', '[FORMULA]', '如图', '[FIGURE]', 'mathord_0', ',', 'mathord_1', '约束条件', '[SEP]', 'mathord_2', '=', '+', 'textord', '最大值', '[MARK]']


In [10]:
tokenizer("公式")

{'input_ids': tensor([[  101, 21138,   102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}

In [11]:
tokenizer("公 式")

{'input_ids': tensor([[ 101, 1062, 2466,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}